In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [2]:
from pinecone import Pinecone
# 클라이언트 생성
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
index_name = "wiki"

# Pinecone에 있는 모든 인덱스를 순회합니다.
for idx in pinecone_client.list_indexes():
    # 인덱스 이름이 "quickstart"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pinecone_client.delete_index(idx.name)

In [4]:
from pinecone import ServerlessSpec
pinecone_client.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-o45zd0f.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [5]:
index = pinecone_client.Index('wiki')
index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [6]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

In [7]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [8]:
len(data)

100

In [9]:
for rec in data:
    if len(rec['text'])>35000:
        print('**')

**
**


In [10]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001E67BA4BB00>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001E67D356510>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 20,
    length_function = len,
    separators=['\n\n','\n',' ', ''] #자르는 순서 문자
)
splitter

In [12]:
import json
def calculate_metadata_size(metadata):
  return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [13]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20

In [ ]:
for _, sample in enumerate(data):
    full_text = sample['text'] # 전문
    # 메타데이터
    metadata = {
        "wiki_id" : str(sample['id']),
        "url" : sample['url'],
        "title" : sample['title']
    }
    # 스플리터로 자르기
    chunks = splitter.split_text(full_text)
    # print(len(chunk))
    texts = []
    metas = []
    # count = 0
    for i, chunk in enumerate(chunks):
        record = {
            'chunk_id':i,
            'full_text':full_text,
            **metadata
        }
        metadata_size = calculate_metadata_size(record)
        if metadata_size > 35000:
            continue
        
        texts.append(chunk)
        metas.append(record)
        count += 1
        
        # 청크 배치의 임베딩 -> 적재
        if count % batch_size == 0:
            ids = [str(uuid4()) for _ in range(len(texts))]
            embeddings = embedding.embed_documents(texts) #20개 청크의 임베딩 수행
            index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace='wiki-ns1'
            )
            time.sleep(1)

# 질의 임베딩

In [30]:
question = ['벨기에는 어디에 있나요?']
emb_question = embedding.embed_documents(question)

In [31]:
response = index.query(
    namespace='wiki-ns1',
    vector=emb_question, # 질의 벡터
    top_k=5, # 몇개 결과
    # include_values=True,
    include_metadata=True
)
response

{'matches': [{'id': '659f13d4-2bf7-49ab-847f-be191a2f637e',
              'metadata': {'chunk_id': 35.0,
                           'full_text': '막스 카를 에른스트 루트비히 플랑크 왕립학회 외국인 회원 (, ; '
                                        ', 1858년 4월 23일 - 1947년 10월 4일)는 독일의 '
                                        '이론물리학자로 에너지 양자의 발견으로 1918년 노벨 물리학상을 '
                                        '수상한 과학자이다.\n'
                                        '\n'
                                        '플랑크는 이론 물리학에 상당히 많은 기여를 했지만 물리학자로서의 '
                                        '명성은 주로 양자 이론의 창시자로서의 역할에 있다. 이는 원자 및 '
                                        '아원자 과정에 대한 인간의 이해에 혁명을 일으켰다. 1948년 '
                                        '독일의 과학 기관인 카이저 빌헬름 협회(플랑크가 두 번 회장을 '
                                        '역임함)는 막스 플랑크 협회(MPG)로 이름이 변경되었다. 이 '
                                        '협회에는 현재 광범위한 과학적 동향을 나타내는 83개의 기관이 '
                                        '포함되어 있다.\n'
                                 

In [32]:
result_ids = [r.id for r in response.matches]
result_ids

['659f13d4-2bf7-49ab-847f-be191a2f637e',
 'd770e202-d640-4ab2-b752-b3e2250598cf',
 'e6da8dde-6966-4216-82bb-1211a8c7f43a',
 'e4d11ddb-341c-4800-ae3d-0edc58041843',
 '132dbf4f-37a7-472d-9ea0-7fdb2244f9a3']

In [33]:
for r in response.matches:
    print(r.score, r.metadata)

0.253605872 {'chunk_id': 35.0, 'full_text': '막스 카를 에른스트 루트비히 플랑크 왕립학회 외국인 회원 (, ; , 1858년 4월 23일 - 1947년 10월 4일)는 독일의 이론물리학자로 에너지 양자의 발견으로 1918년 노벨 물리학상을 수상한 과학자이다.\n\n플랑크는 이론 물리학에 상당히 많은 기여를 했지만 물리학자로서의 명성은 주로 양자 이론의 창시자로서의 역할에 있다. 이는 원자 및 아원자 과정에 대한 인간의 이해에 혁명을 일으켰다. 1948년 독일의 과학 기관인 카이저 빌헬름 협회(플랑크가 두 번 회장을 역임함)는 막스 플랑크 협회(MPG)로 이름이 변경되었다. 이 협회에는 현재 광범위한 과학적 동향을 나타내는 83개의 기관이 포함되어 있다.\n\n생애와 경력 \n\n플랑크는 전통적이고 지적인 가족 출신이다. 그의 부계 증조부와 할아버지는 모두 괴팅겐의 신학 교수였다. 그의 아버지는 킬 대학교와 뮌헨 대학교의 법학 교수였다. 그의 삼촌 중 한 명은 판사였다.\n\n플랑크는 1858년 홀슈타인의 킬에서 요한 율리우스 빌헬름 플랑크Johann Julius Wilhelm Planck와 그의 두 번째 부인 엠마 파치히Emma Patzig 사이에서 태어났다. 그는 카를 에른스트 루트비히 마르크스 플랑크Karl Ernst Ludwig Marx Planck라는 이름으로 세례를 받았다. 그의 이름 중 마르크스Marx(지금은 사용되지 않는 마르쿠스Markus의 변형이거나 실제로 막시밀리안Maximilian의 약자인 막스Max의 오류일 수 있음)는 "명칭 이름(appellation name)"으로 표시되었다. 그러나 10살이 되었을 때 그는 막스Max라는 이름으로 서명하고 그후 이것을 평생동안 사용했다.\n\n그의 형제 중 두 명이 아버지의 첫 번째 결혼에서 태어났지만 그는 가족의 6번째였다. 전쟁은 플랑크의 초기 몇 년 동안 일반적이었고 그의 가장 초기 기억 중 하나는 1864년 제2차 슐레스비히 전쟁 동안 킬로의 프로이센과 오스

# 랭체인의 api로 결과 비교하기

In [34]:
question

['벨기에는 어디에 있나요?']

In [35]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

[Document(id='659f13d4-2bf7-49ab-847f-be191a2f637e', metadata={'chunk_id': 35.0, 'title': '막스 플랑크', 'url': 'https://ko.wikipedia.org/wiki/%EB%A7%89%EC%8A%A4%20%ED%94%8C%EB%9E%91%ED%81%AC', 'wiki_id': '228'}, page_content='막스 카를 에른스트 루트비히 플랑크 왕립학회 외국인 회원 (, ; , 1858년 4월 23일 - 1947년 10월 4일)는 독일의 이론물리학자로 에너지 양자의 발견으로 1918년 노벨 물리학상을 수상한 과학자이다.\n\n플랑크는 이론 물리학에 상당히 많은 기여를 했지만 물리학자로서의 명성은 주로 양자 이론의 창시자로서의 역할에 있다. 이는 원자 및 아원자 과정에 대한 인간의 이해에 혁명을 일으켰다. 1948년 독일의 과학 기관인 카이저 빌헬름 협회(플랑크가 두 번 회장을 역임함)는 막스 플랑크 협회(MPG)로 이름이 변경되었다. 이 협회에는 현재 광범위한 과학적 동향을 나타내는 83개의 기관이 포함되어 있다.\n\n생애와 경력 \n\n플랑크는 전통적이고 지적인 가족 출신이다. 그의 부계 증조부와 할아버지는 모두 괴팅겐의 신학 교수였다. 그의 아버지는 킬 대학교와 뮌헨 대학교의 법학 교수였다. 그의 삼촌 중 한 명은 판사였다.\n\n플랑크는 1858년 홀슈타인의 킬에서 요한 율리우스 빌헬름 플랑크Johann Julius Wilhelm Planck와 그의 두 번째 부인 엠마 파치히Emma Patzig 사이에서 태어났다. 그는 카를 에른스트 루트비히 마르크스 플랑크Karl Ernst Ludwig Marx Planck라는 이름으로 세례를 받았다. 그의 이름 중 마르크스Marx(지금은 사용되지 않는 마르쿠스Markus의 변형이거나 실제로 막시밀리안Maximilian의 약자인 막스Max의 오류일 수 있음)는 "명칭 이름(appellation name)"으로 표시되었다. 그러나 10

In [36]:
questions = [
    '류노스케씨는 누구인가?',
    '함석헌씨는 누구인가?',
    '각 나라의 수도는 어디인가?'
]

In [40]:
emb_questions = embedding.embed_documents(questions)

In [47]:
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=questions[0], k=5, namespace='wiki-ns1')
docs

[Document(id='96b16ca5-341c-4be8-b6e8-7da5cd6a2129', metadata={'chunk_id': 18.0, 'title': '빅토르 초이', 'url': 'https://ko.wikipedia.org/wiki/%EB%B9%85%ED%86%A0%EB%A5%B4%20%EC%B4%88%EC%9D%B4', 'wiki_id': '195'}, page_content='빅토르 로베르토비치 초이(, 1962년 6월 21일 ~ 1990년 8월 15일)는 소련의 록 가수이자, 싱어송라이터 겸 배우이며, 소련 록 음악 밴드 키노(КИНО)의 리더였다.\n\n생애\n\n데뷔 이전 \n빅토르 초이는 1962년 6월 21일, 소련 레닌그라드에서 아버지 로베르트 막시모비치 초이(최동열)와 우크라이나계 러시아인 출신 어머니 사이에서 슬하 무녀독남 외동아들로 출생하였다. 친조부 막심 초이(최승준)는 본래 대한제국 함경북도 성진 출생이었고 후일 일제 강점기 초기에 러시아 제국으로 건너간 고려인 출신이었다. 소련 레닌그라드에서 출생하였으며 지난날 한때 소련 카자흐스탄 사회주의 자치공화국 키질로르다에서 잠시 유아기를 보낸 적이 있는 그는 17세 때부터 노래를 작곡하기 시작했으며, 초기 곡들은 레닌그라드 거리에서의 삶, 사랑과 친구들과의 어울림 등을 다루고 있다. 노래의 주인공은 주로 한정된 기회만이 주어진 채 각박한 세상을 살아나가려는 젊은이였다. 이 시기에 록은 레닌그라드에서만 태동하고 있던 언더그라운드의 한 움직임이었으며, 음악 차트 등의 대중 매체들은 모스크바의 팝 스타들이 장악하고 있었다. 더군다나 소련 정부는 자신들의 입맛에 맞는 가수들에게만 허가를 내 주었고, 집과 녹음실 등 성공이 필요한 많은 것들을 제공하여 길들였다. 그러나 록 음악은 그 당시 소련 정부에게 너무도 마땅치 않은 음악이었다. 록은 자본주의 진영의 록 그룹의 영향을 받았다는 것 외에도 젊은이들을 반항적으로 만들었으며, 의사 표현의 자유 등 표현 관련 가치를 중시했다. 따라서 록 밴

In [46]:
response = index.query(
    namespace='wiki-ns1',
    vector=emb_questions[0], # 질의 벡터
    top_k=5, # 몇개 결과
    # include_values=True,
    include_metadata=True
)
for r in response.matches:
    print(r.score, r.metadata)

0.377822876 {'chunk_id': 18.0, 'full_text': '빅토르 로베르토비치 초이(, 1962년 6월 21일 ~ 1990년 8월 15일)는 소련의 록 가수이자, 싱어송라이터 겸 배우이며, 소련 록 음악 밴드 키노(КИНО)의 리더였다.\n\n생애\n\n데뷔 이전 \n빅토르 초이는 1962년 6월 21일, 소련 레닌그라드에서 아버지 로베르트 막시모비치 초이(최동열)와 우크라이나계 러시아인 출신 어머니 사이에서 슬하 무녀독남 외동아들로 출생하였다. 친조부 막심 초이(최승준)는 본래 대한제국 함경북도 성진 출생이었고 후일 일제 강점기 초기에 러시아 제국으로 건너간 고려인 출신이었다. 소련 레닌그라드에서 출생하였으며 지난날 한때 소련 카자흐스탄 사회주의 자치공화국 키질로르다에서 잠시 유아기를 보낸 적이 있는 그는 17세 때부터 노래를 작곡하기 시작했으며, 초기 곡들은 레닌그라드 거리에서의 삶, 사랑과 친구들과의 어울림 등을 다루고 있다. 노래의 주인공은 주로 한정된 기회만이 주어진 채 각박한 세상을 살아나가려는 젊은이였다. 이 시기에 록은 레닌그라드에서만 태동하고 있던 언더그라운드의 한 움직임이었으며, 음악 차트 등의 대중 매체들은 모스크바의 팝 스타들이 장악하고 있었다. 더군다나 소련 정부는 자신들의 입맛에 맞는 가수들에게만 허가를 내 주었고, 집과 녹음실 등 성공이 필요한 많은 것들을 제공하여 길들였다. 그러나 록 음악은 그 당시 소련 정부에게 너무도 마땅치 않은 음악이었다. 록은 자본주의 진영의 록 그룹의 영향을 받았다는 것 외에도 젊은이들을 반항적으로 만들었으며, 의사 표현의 자유 등 표현 관련 가치를 중시했다. 따라서 록 밴드들은 정부로부터 거의 원조를 받지 못했고 관영 매체에 의해 마약 중독자나 부랑자라는 편견으로 그려지는 수준이었다.\n\n빅토르 초이는 레닌그라드에 있는 세로프 미술전문학교에 입학였으나, 결국 낮은 성적 때문에 1977년에 퇴학 처분을 받았다. 그 후 레닌그라드 기술전문학교에서 목공업을 공부하였으나, 적성에 맞지 않